In [1]:
import sys
print(sys.executable)

c:\Users\DELL\AppData\Local\Programs\Python\Python310\python.exe


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import csv
import time
from bs4 import BeautifulSoup
import requests

In [2]:
def chrome():
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-ssl-errors=yes')
    options.add_argument('--ignore-certificate-errors')
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    return driver
driver = chrome()
urls = []

website = 'https://annuairesante.ameli.fr/'
path = 'C:/Users/DELL/Desktop/MediConnect/chrome-win64/chrome-win64/chrome.exe'
#driver = webdriver.Chrome(path)
driver.get(website)

In [3]:
list_profession = driver.find_element("css selector", ".popin-autocomplete-ps-professions.popin-autocomplete-enable")
list_profession.click()


In [4]:
index = driver.find_element("id" , "index-0").find_element("tag name" , "a")
index.click()

In [5]:
submit = driver.find_element("name" , "submit_final")
submit.click()

In [6]:
professionnels = driver.find_element("class name" , "liste-professionnel")

In [7]:
professionnel = driver.find_element("class name" , "item-professionnel").find_element("class name" , "item-professionnel-inner")

In [8]:
# extract each doctor id 
num_spans = driver.find_elements("css selector", ".item-professionnel .num")
#print(num_span.text)

In [9]:
for num_s in num_spans:
    try:
        parent_div = num_s.find_element("xpath", "./ancestor::div[@class='item-professionnel']")
        nom_pictos = parent_div.find_element("css selector", ".nom_pictos h2 a")
        #print(nom_pictos.text)

        last_name = nom_pictos.find_element("css selector" , "strong").text
        firstname = nom_pictos.text.replace(last_name, "").strip()

        print(num_s.text)
        print(last_name)
        print(firstname)

    except Exception as e:
        print("Error processing:", e)
    


1
HANNOUN
SYLVIE
2
BEGHI
PASCAL
3
SAUVEGRAIN MASSIN
ISABELLE
4
BOUCHOUCHA
MARC
5
COHEN
MICHELE
6
LECOLLIER
DIDIER
7
LABORIE
GILLES
8
VU
GIANG
9
ATHLAN CHELLY
FREDERIQUE
10
THOUMMANY
FRANCE ARMELLE
11
BURGOS CANTOR
JAVIER
12
LEBORGNE
GIL
13
FRISON
CHRISTIAN
14
BECK
ANDRE
15
DELAS
GEORGES
16
BOURLEAUD
ALAIN
17
MILBERT
BEATRICE
18
FAURE ANTONIETTI
FREDERIQUE
19
MINIOU
VERONIQUE
20
FORSANS
ALAIN


In [10]:
elements = driver.find_elements("css selector" , "elements")

In [15]:
from time import sleep
sleep(2)

for element in elements:
    try:
        specialite = element.find_element("css selector", ".item.left.specialite")
        print(specialite.text)

        tel = element.find_element("css selector" , ".item left tel")
        print(tel.text)

        address = element.find_element("css selector" , ".item left adresse")
        print(address.text)

        sleep(1)
        

    except Exception as e:
        print("Error processing:", e)


In [ ]:
for i in range(92):
    index = driver.find_element("id", "index-{i}")

In [ ]:
#beautifulsoup
with open("liste_des_medecins.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file, quotechar='"', quoting= csv.QUOTE_MINIMAL)
    writer.writerow(["Nom", "Prénom", "Spécialité", "Clinique","Rue", "Ville", "Code Postal", "Numéro de téléphone", "Email"])

    
    for page in range(1,21): 
        url = "https://annuairesante.ameli.fr/professionnels-de-sante/recherche/liste-resultats-page-{page}-par_page-20-tri-aleatoire.html"
        print(f"Scraping: {url}")

        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text,"html.parser")

        if response.status_code != 200:
            print(f"❌ Error {response.status_code}: Stopping scrape.")
            break
        
        professionals = soup.find_all('div', class_="liste-professionnel")
        print(professionals)
            
#professionals = soup.select(".liste-professionnel .item-professionnel")
        print("Total professionals found:", len(professionals))
        for pro in professionals:
            print(pro.text.strip())  # Print the content of each professional
        if not professionals:
            print(f"No professionals found on page {page}. Continuing...")
            break

        for professional in professionals:
            inner_pro = professional.find("div", class_="item-professionnel-inner")
            if not inner_pro:
                continue 

            nomc = inner_pro.find("h2", class_="ignore-css")
            if nomc:
                nom = nomc.find("strong").text.strip() if nomc.find("strong") else "NA"
                prenom = nomc.get_text(strip=True).replace(nom, "").strip()
            else:
                    nom, prenom = "NA", "NA"

            elements = inner_pro.find("div", class_="elements")
            if not elements:
                continue

            specialite = elements.find("div", class_="item left specialite")
            specialite = specialite.get_text(strip=True) if specialite else "NA"

            address = elements.find("div", class_="item left adresse")
            if address:
                full_address = address.get_text(separator="|", strip=True).split("|") 

                if len(full_address) == 3:
                    clinic = full_address[0]
                    street = full_address[1]
                    city_postal = full_address[2]
                elif len(full_address) == 2:
                    clinic = "NA"
                    street = full_address[0]
                    city_postal = full_address[1]
                else:
                    clinic, street, city_postal = "NA"

                # Separate postal code and city
                code_postal, city = city_postal.split(" ", 1) if " " in city_postal else ("N/A", city_postal)
            else:
                clinic, street, city, code_postal = "N/A", "N/A", "N/A", "N/A"

            tel_div = elements.find("div", class_="item left tel")
            tel = tel_div.get_text(strip=True) if tel_div else "NA"

            email_div = elements.find("div", class_="item left email")
            email = email_div.get_text(strip=True) if email_div else "NA"
                    
            writer.writerow([nom, prenom, specialite, clinic, street, city, code_postal, tel, email])

        #page += 1

        print("✅ Scraping completed. Data saved to 'liste_des_medecins.csv'.")

            
        

In [9]:
print(soup.prettify())  # See if the HTML is correct


<html>
 <head>
  <title>
   Request Rejected
  </title>
 </head>
 <body>
  The requested URL was rejected. Please consult with your administrator.
  <br/>
  <br/>
  Your support ID is: 8402854357907416372
  <br/>
  <br/>
  <a href="javascript:history.back();">
   [Go Back]
  </a>
 </body>
</html>

